# Eleos

There was in the midst of Athens an altar belonging to no god of power; gentle Eleos had there her seat, and the wretched made it sacred; never lacked she a new suppliant, none did she condemn or refuse their prayers. All that ask are heard, night and day may one approach and win the heart of the goddess by complaints alone. No costly rites are hers; she accepts no incense flame, no blood deep-welling; tears flow upon her altar, sad offering of severed tresses hang above it, and raiment left when fortune changed. Around is a grove of gentle trees, marked by the cult of the venerable, wool-entwined laurel and the suppliant olive. No image is there, to no metal is the divine form entrusted, in hearts and minds does the goddess delight to dwell. The distressed are ever nigh her, her precinct ever swarms with needy folk, only to the prosperous her shrine is unknown. 

In [ ]:
# dependencies
import json
import tweepy
import pymongo
import gspread
import time
import pandas as pd
import numpy as np
from pymongo import MongoClient
from oauth2client.service_account import ServiceAccountCredentials


# Specify twitter account credentials in the following variables:
with open('Authorization/twitter_client_secret.json') as json_key:
    twitter_key = json.load(json_key)

consumer_key = twitter_key['consumer_key']
consumer_secret = twitter_key['consumer_secret']
access_token = twitter_key['access_token']
access_token_secret = twitter_key['access_token_secret']

#authorize twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Specify google account credentials in the following variables:
with open('Authorization/google_client_secret.json') as json_key:
    google_key = json.load(json_key)
API_KEY = google_key['private_key']

#authorize google sheets
scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('Authorization/google_client_secret.json', scope)
gc = gspread.authorize(credentials)

#Mongodb location
mongo_Host = 'mongodb://localhost:27017/' 


In [ ]:
#Functions & Classes

#Find ID from user handle
def find_id(name_list):
    for name in handles:
        print('Retrieving '+name+'\'s id')
        while True:
            try:
                user = api.get_user(name)
                ids.append(str(user.id))
            #in the case of rate limit reached, wait until reset and continue
            except tweepy.RateLimitError:
                print('Rate limit reached, waiting 15 minutes...')
                time.sleep(915)
                continue
            except tweepy.TweepError:
                print(name + ' not found')
            break
        print('All id\'s have been retrieved...')
            
#initialize the api object
class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 
 
    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        #This is the meat of the script...it connects to your mongoDB and stores the tweet
        try:
            client = MongoClient(mongo_Host)
            
            # Use database. If it doesn't exist, it will be created.
            db = client.meantweets
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']
 
            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            
            #insert the data into the mongoDB into a collection 
            #if twitter_search doesn't exist, it will be created.
            db.pre_election.insert_one(datajson)
        except Exception as e:
           print(e)
        
def start_stream():
    print('Eleos is listening...')
    #Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
    listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
    streamer = tweepy.Stream(auth=auth, listener=listener)
    streamer.filter(follow=ids, is_async=True)

In [ ]:
#data inputs

#retrieve a static copy of 'seeds' spreadsheet
wks = gc.open("seeds_static").worksheet('candidates')

#change worksheet object into dataframe
data = wks.get_all_records()
candidates = pd.DataFrame(data, columns=data[0].keys())

#clean for twitter handles
handles = candidates['Twitter'].replace(r'^\s*$', np.nan, regex=True).dropna()

#call the API for ids
api = tweepy.API(auth)
ids = []
find_id(handles)

#cache ids once retreived
ids = pd.DataFrame(ids)
ids.to_csv('Data/Input/user_ids.csv', index=False, header=False)

In [ ]:
#Load data
ids = pd.read_csv('Data/Input/user_ids.csv', header=None, dtype=str)
ids = list(ids[0])

In [ ]:
#Execute Eleos
start_stream()